In [5]:
from dodfminer import ContentExtractor
from dodfminer import ActsExtractor
from dodfminer import downloader
from dodfminer.downloader.core import Downloader
import pandas as pd
import numpy as np
import glob
import re
import os

import nltk

import scipy.stats

import sklearn
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV

import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics

import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

import matplotlib.pyplot as plt
import seaborn as sns
sns.set(color_codes=True)
sns.set(font_scale=1)
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

## Extraindo novos dados

In [6]:
# downloader = Downloader(save_path='./')
# downloader.pull(start_date="09/2021", end_date="10/2021")

In [7]:
# pdf_files = glob.glob('dodfs/**/**/*.pdf')

In [8]:
# for pdf in pdf_files:
#     ContentExtractor.extract_text(pdf, single=True, block=False, sep='\n<END_OF_BLOCK>\n')

In [105]:
txt_files = glob.glob('dodfs/**/**/*.txt')

In [260]:
COLUNAS = [
    "DODF",
    "Texto",
]

data = pd.DataFrame(columns=COLUNAS)
data

,DODF,Texto


In [261]:
for txt in txt_files:
    df = ActsExtractor.get_act_df("licitacao_abertura", txt, 'regex')
    for idx in range(len(df)):
        df['DODF'] = txt[:-4]
        df['Redator'][idx] = re.findall(r"<END_OF_BLOCK>[\s\S][A-Z \\n]+[\s\S]<END_OF_BLOCK>", df['Texto'][idx])[0]
        if pd.isna(df['Data Ato/DODF'][idx]):
            df['Data Ato/DODF'][idx] = re.findall(r'\d{2}-\d{2}-\d{2,4}', df['DODF'][idx])[0]
            if len(df['Data Ato/DODF'][idx]) == 8:
                df['Data Ato/DODF'][idx] = df['Data Ato/DODF'][idx][:6] + '20' + df['Data Ato/DODF'][idx][6:]
    data = pd.concat([data, df], ignore_index=True)

for idx in range(len(data)):
    data['Texto'][idx] = data['Tipo do Ato'][idx] + data['Texto'][idx]
    data['Texto'][idx] = data['Texto'][idx].replace('\n<END_OF_BLOCK>\n', ' ').replace('\n<END_OF_BLOCK>', ' ').replace('  ', ' ')
    data['Texto'][idx] = data['Texto'][idx].replace('\n', ' ').replace('Processo:', 'Processo: ').replace('  ', ' ').replace('- ', '')
    

def clean_text():
    for idx in range(len(data)):
        data['Texto'][idx] =  re.sub(u'[^a-zA-Z0-9áéíóúÁÉÍÓÚâêîôÂÊÎÔãõÃÕçÇ:, ]', '', data['Texto'][idx])

clean_text()


data = data[['Texto']]

C:\Users\ianfi\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Users\ianfi\AppData\Roaming\Python\Python36\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
C:\Users\ianfi\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a

## Tranformando os dados no formato aceito pelo modelo

In [262]:
data

,Texto
0,AVISO DE ABERTURA DE LICITACAO PREGAO ELETRON...
1,AVISO DE ABERTURA DE LICITACAO O PRESIDENTE DA...
2,AVISO DE ABERTURA DE LICITACAO PREGAO ELETRONI...
3,AVISO DE ABERTURA DE LICITACAO O Presidente da...
4,AVISO DE ABERTURA DE LICITACAO PREGAO ELETRONI...
5,AVISO DE ABERTURA DE LICITACAO No 322021 UASG ...
6,AVISO DE ABERTURA DE LICITACAO PREGAO ELETRONI...
7,AVISO DE ABERTURA DE LICITACAO O Presidente da...
8,AVISO DE ABERTURA DE LICITACAO O Presidente da...
9,AVISO DE ABERTURA DE LICITACAO SRP PREGAO ELET...


In [263]:
DEFAULT_TOKENIZER = nltk.RegexpTokenizer(r"\w+").tokenize

x = []
y = []
for row in range(len(data)):
    if pd.notna(data['Texto'][row]):
        x.append(DEFAULT_TOKENIZER(data['Texto'][row]))
        y.append(DEFAULT_TOKENIZER(data['Texto'][row]))

In [264]:
def get_features(sentence):
        """Create features for each word in act.
        Create a list of dict of words features to be used in the predictor module.
        Args:
            act (list): List of words in an act.
        Returns:
            A list with a dictionary of features for each of the words.
        """
        sent_features = []
        for i in range(len(sentence)):
            word_feat = {
                'word': sentence[i].lower(),
                'capital_letter': sentence[i][0].isupper(),
                'all_capital': sentence[i].isupper(),
                'isdigit': sentence[i].isdigit(),
                'word_before': sentence[i].lower() if i == 0 else sentence[i-1].lower(),
                'word_after:': sentence[i].lower() if i+1 >= len(sentence) else sentence[i+1].lower(),
                'BOS': i == 0,
                'EOS': i == len(sentence)-1
            }
            sent_features.append(word_feat)
        return sent_features
    
for i in range(len(x)):
    x[i] = get_features(x[i])

In [265]:
x[0]

[{'word': 'aviso',
  'capital_letter': True,
  'all_capital': True,
  'isdigit': False,
  'word_before': 'aviso',
  'word_after:': 'de',
  'BOS': True,
  'EOS': False},
 {'word': 'de',
  'capital_letter': True,
  'all_capital': True,
  'isdigit': False,
  'word_before': 'aviso',
  'word_after:': 'abertura',
  'BOS': False,
  'EOS': False},
 {'word': 'abertura',
  'capital_letter': True,
  'all_capital': True,
  'isdigit': False,
  'word_before': 'de',
  'word_after:': 'de',
  'BOS': False,
  'EOS': False},
 {'word': 'de',
  'capital_letter': True,
  'all_capital': True,
  'isdigit': False,
  'word_before': 'abertura',
  'word_after:': 'licitacao',
  'BOS': False,
  'EOS': False},
 {'word': 'licitacao',
  'capital_letter': True,
  'all_capital': True,
  'isdigit': False,
  'word_before': 'de',
  'word_after:': 'pregao',
  'BOS': False,
  'EOS': False},
 {'word': 'pregao',
  'capital_letter': True,
  'all_capital': True,
  'isdigit': False,
  'word_before': 'licitacao',
  'word_after:': 

## Testando o modelo

In [266]:
import joblib
model = joblib.load('crf_model.pkl')

In [267]:
classes = list(model.classes_)
classes.remove('O')

y_pred = model.predict(x)
len(y_pred), len(x), len(data['Texto'])

(32, 32, 32)

In [268]:
len(y_pred[0]), len(y[0])

(206, 206)

In [269]:
model.classes_

['B-ATO',
 'I-ATO',
 'O',
 'B-DATA',
 'I-DATA',
 'B-PROCESSO',
 'B-REDATOR',
 'I-REDATOR',
 'B-V_ESTIMADO',
 'I-V_ESTIMADO',
 'B-TIPO',
 'I-TIPO',
 'B-DISPUTA',
 'I-DISPUTA']

In [270]:
data['IOB'] = np.nan
for idx in range(len(data)):
    data['IOB'][idx] = ' '.join(y_pred[idx])

C:\Users\ianfi\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\ianfi\AppData\Roaming\Python\Python36\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [271]:
data

,Texto,IOB
0,AVISO DE ABERTURA DE LICITACAO PREGAO ELETRON...,B-ATO I-ATO I-ATO I-ATO I-ATO O O O O O B-PROC...
1,AVISO DE ABERTURA DE LICITACAO O PRESIDENTE DA...,B-ATO I-ATO I-ATO I-ATO I-ATO O O O O O O O O ...
2,AVISO DE ABERTURA DE LICITACAO PREGAO ELETRONI...,B-ATO I-ATO I-ATO I-ATO I-ATO O O O O O O O B-...
3,AVISO DE ABERTURA DE LICITACAO O Presidente da...,B-ATO I-ATO I-ATO I-ATO I-ATO O O O O O O O O ...
4,AVISO DE ABERTURA DE LICITACAO PREGAO ELETRONI...,B-ATO I-ATO I-ATO I-ATO I-ATO O O O O O O O O ...
5,AVISO DE ABERTURA DE LICITACAO No 322021 UASG ...,B-ATO I-ATO I-ATO I-ATO I-ATO O O O O O O O O ...
6,AVISO DE ABERTURA DE LICITACAO PREGAO ELETRONI...,B-ATO I-ATO I-ATO I-ATO I-ATO O O O O O B-PROC...
7,AVISO DE ABERTURA DE LICITACAO O Presidente da...,B-ATO I-ATO I-ATO I-ATO I-ATO O O O O O O O O ...
8,AVISO DE ABERTURA DE LICITACAO O Presidente da...,B-ATO I-ATO I-ATO I-ATO I-ATO O O O O O O O O ...
9,AVISO DE ABERTURA DE LICITACAO SRP PREGAO ELET...,B-ATO I-ATO I-ATO I-ATO I-ATO O O O O O O O O ...


## Gerando as colunas com as entidades encontradas pelo modelo

In [272]:
data['ATO'] = np.nan
data['PROCESSO'] = 'nan'

for idx in range(len(data)):
    tex = DEFAULT_TOKENIZER(data['Texto'][idx])
    iob = data['IOB'][idx].split()

    if 'B-ATO' in iob and 'I-ATO' in iob:
        data['ATO'][idx] = ' '.join(tex[iob.index('B-ATO'):iob.index('I-ATO', 4)+1])

    if 'B-PROCESSO' in iob:
        data['PROCESSO'][idx] = tex[iob.index('B-PROCESSO')]

C:\Users\ianfi\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
C:\Users\ianfi\AppData\Roaming\Python\Python36\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


## Dataframe gerado

In [278]:
data

,Texto,IOB,ATO,PROCESSO
0,AVISO DE ABERTURA DE LICITACAO PREGAO ELETRON...,B-ATO I-ATO I-ATO I-ATO I-ATO O O O O O B-PROC...,AVISO DE ABERTURA DE LICITACAO,0402600012872202106
1,AVISO DE ABERTURA DE LICITACAO O PRESIDENTE DA...,B-ATO I-ATO I-ATO I-ATO I-ATO O O O O O O O O ...,AVISO DE ABERTURA DE LICITACAO,0011100016333201703
2,AVISO DE ABERTURA DE LICITACAO PREGAO ELETRONI...,B-ATO I-ATO I-ATO I-ATO I-ATO O O O O O O O B-...,AVISO DE ABERTURA DE LICITACAO,0040100008173202014
3,AVISO DE ABERTURA DE LICITACAO O Presidente da...,B-ATO I-ATO I-ATO I-ATO I-ATO O O O O O O O O ...,AVISO DE ABERTURA DE LICITACAO,0011100000404202124
4,AVISO DE ABERTURA DE LICITACAO PREGAO ELETRONI...,B-ATO I-ATO I-ATO I-ATO I-ATO O O O O O O O O ...,AVISO DE ABERTURA DE LICITACAO,0006400003361202114
5,AVISO DE ABERTURA DE LICITACAO No 322021 UASG ...,B-ATO I-ATO I-ATO I-ATO I-ATO O O O O O O O O ...,AVISO DE ABERTURA DE LICITACAO,0006300003288202117
6,AVISO DE ABERTURA DE LICITACAO PREGAO ELETRONI...,B-ATO I-ATO I-ATO I-ATO I-ATO O O O O O B-PROC...,AVISO DE ABERTURA DE LICITACAO,00031571202061
7,AVISO DE ABERTURA DE LICITACAO O Presidente da...,B-ATO I-ATO I-ATO I-ATO I-ATO O O O O O O O O ...,AVISO DE ABERTURA DE LICITACAO,0011100000502202161
8,AVISO DE ABERTURA DE LICITACAO O Presidente da...,B-ATO I-ATO I-ATO I-ATO I-ATO O O O O O O O O ...,AVISO DE ABERTURA DE LICITACAO,0011100007409202005
9,AVISO DE ABERTURA DE LICITACAO SRP PREGAO ELET...,B-ATO I-ATO I-ATO I-ATO I-ATO O O O O O O O O ...,AVISO DE ABERTURA DE LICITACAO,0402600000290202179


In [279]:
data.isnull().sum()

Texto       0
IOB         0
ATO         0
PROCESSO    0
dtype: int64

In [ ]:
# pytorch serve - gera api - pesquisar